In [15]:
#Inspired by fast.ai Practical Deep Learning for Coders(v3) lession 1

In [16]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [17]:
!pip freeze > requirements.txt

In [18]:
!pip --version

In [19]:
from fastai.vision import *
from fastai.metrics import error_rate

In [20]:
bs = 64

In [22]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
print(os.listdir('../input/final-car-training-dataset/car_data/train'))

In [24]:
data_dir='../input/final-car-training-dataset/car_data/train'

list = os.listdir(data_dir) 
number_files = len(list)
print(number_files)

In [25]:
path=Path(data_dir)
path

In [26]:
data = ImageDataBunch.from_folder(path,  
                                  valid_pct=0.2,
                                  ds_tfms=get_transforms(do_flip=True,flip_vert=False, max_rotate=90),
                                  size=224,bs=64, 
                                  num_workers=0).normalize(imagenet_stats)

In [ ]:
test_data = ImageDataBunch.from_folder("../input/final-car-train/car_data/test",  
                                  valid_pct=0,
                                  ds_tfms=get_transforms(do_flip=False,flip_vert=False, max_rotate=0),
                                  size=224,bs=64, 
                                  num_workers=0).normalize(imagenet_stats)

In [27]:
data.show_batch(rows=3, figsize=(40,40))

In [28]:
print(data.classes)
len(data.classes)

In [29]:
learn = create_cnn(data, models.resnet34, metrics=accuracy, model_dir="/tmp/model/")

In [30]:
learn.model

In [31]:
learn.fit_one_cycle(10)

In [32]:
learn.save('stage-1')

In [33]:
interp = ClassificationInterpretation.from_learner(learn)

losses,idxs = interp.top_losses()

len(data.valid_ds)==len(losses)==len(idxs)

In [34]:
interp.plot_top_losses(9, figsize=(30,25))

In [35]:
interp.most_confused(min_val=2)

Unfreezing, fine-tuning, and learning rates

In [36]:
learn.unfreeze()

In [37]:
learn.fit_one_cycle(10)

In [38]:
learn.lr_find()

In [39]:

learn.recorder.plot()

In [40]:
learn.unfreeze()
learn.fit_one_cycle(4, max_lr=slice(1e-6,1e-4))

In [41]:
learn.save('stage-2')

In [42]:
interp = ClassificationInterpretation.from_learner(learn)

In [43]:
losses,idxs = interp.top_losses()

len(data.valid_ds)==len(losses)==len(idxs)

In [44]:
interp.plot_top_losses(9, figsize=(30,25))

In [48]:
#image = Image.open("../input/final-car-train/car_data/test/AM General Hummer SUV 2000/00076.jpg")
#x = pil2tensor(image, np.float32)
#preds_num = learn.predict(Image(x))[2].numpy()

#trn_tfms, val_tfms = tfms_from_model(arch,sz) # get transformations
im = open_image("../input/finalpredictcars/predict/10190807793.jpg")
#learn.precompute=False # We'll pass in a raw image, not activations
#reds = learn.predict_array(im[None])
#p.argmax(preds) # preds are log probabilities of classes

pred_class, pred_idx, outputs =   learn.predict(im)

In [ ]:
print(pred_class)

In [83]:
footprints = pd.read_csv('../input/cars-csv/car_models_footprint.csv',  sep=";")

In [50]:
intermediate = pd.read_csv('../input/intermediatesolution/intermediate_solution.csv')

In [ ]:
directory = '../input/finalpredictcars/predict'

prediction = []

for i in range(len(intermediate)):
    filename = intermediate.iloc[i]['im_name']
    im = open_image("../input/finalpredictcars/predict/" + filename)
    pred_class, pred_idx, outputs = learn.predict(im)
    intermediate.loc[i, 'class_pred'] = pred_class

In [62]:
print(intermediate.head())

In [ ]:
for i in range(len(intermediate)):
        if(intermediate.iloc[i]['classType'] == 0.0):
            intermediate.loc[i, 'e'] = 0

In [ ]:
for i in range(len(intermediate)):
    if(intermediate.iloc[i]['classType'] == 1.0):
        intermediate.loc[i, 'e'] = footprints.loc[footprints['models'] == str(intermediate.iloc[i]['class_pred']), 'Average of CO2 (g per km)'].values[0]

In [109]:
final = intermediate[['im_name','x_min','y_min','x_max','y_max','e']]

In [112]:
print(final)

In [113]:
final.to_csv('boostrapers.csv', index = False, header=True)